# Importing libraries

In [2]:
from typing import List
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets, layers, models

import weight_manager

import cloudpickle

import pathlib
# https://www.geeksforgeeks.org/ordereddict-in-python/

# Creating model 

In [3]:
# simple model creation

model = tf.keras.Sequential()
model.add(layers.Dense(units=10, input_shape=(1,)))
model.add(layers.Dense(units=10))
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32,32,3)))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Conv2D(64, (3, 3), activation='relu'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


# Finding how to save and load the differents parameters of a `tf.keras.Sequential` to accomplish the aggregation phase of the Substrafl experiment

In [4]:
# automatic name creation
model.layers[0].name,model.layers[1].name

('dense', 'dense_1')

In [5]:
model.layers[0].get_weights() # gives (weights, bias)

[array([[-0.49551314,  0.35221773,  0.5255818 ,  0.15151203,  0.44275302,
          0.7328324 ,  0.0546217 ,  0.16471899,  0.22105348,  0.02140474]],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)]

In [6]:
f'{2+2}'

'4'

In [7]:
weight_manager.model_state_dict(model)

OrderedDict([('config',
              {'name': 'sequential',
               'layers': [{'class_name': 'InputLayer',
                 'config': {'batch_input_shape': (None, 1),
                  'dtype': 'float32',
                  'sparse': False,
                  'ragged': False,
                  'name': 'dense_input'}},
                {'class_name': 'Dense',
                 'config': {'name': 'dense',
                  'trainable': True,
                  'dtype': 'float32',
                  'batch_input_shape': (None, 1),
                  'units': 10,
                  'activation': 'linear',
                  'use_bias': True,
                  'kernel_initializer': {'class_name': 'GlorotUniform',
                   'config': {'seed': None}},
                  'bias_initializer': {'class_name': 'Zeros', 'config': {}},
                  'kernel_regularizer': None,
                  'bias_regularizer': None,
                  'activity_regularizer': None,
                  'ke

In [8]:
optimizer.get_config() # OK, et 'state ?'

{'name': 'Adam',
 'weight_decay': None,
 'clipnorm': None,
 'global_clipnorm': None,
 'clipvalue': None,
 'use_ema': False,
 'ema_momentum': 0.99,
 'ema_overwrite_frequency': None,
 'jit_compile': False,
 'is_legacy_optimizer': False,
 'learning_rate': 0.001,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'epsilon': 1e-07,
 'amsgrad': False}

In [9]:
model.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_input'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 1),
    'units': 10,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_init

In [10]:
model.get_weights() #list

[array([[-0.49551314,  0.35221773,  0.5255818 ,  0.15151203,  0.44275302,
          0.7328324 ,  0.0546217 ,  0.16471899,  0.22105348,  0.02140474]],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([[ 0.41724896,  0.45191437,  0.340787  ,  0.5463778 , -0.38249683,
         -0.23618022, -0.18816003, -0.42619744, -0.00390798,  0.07460922],
        [-0.15487856,  0.39861256, -0.5139727 , -0.30955482,  0.31888008,
         -0.30110312, -0.4486343 , -0.41079572, -0.44411612,  0.3140511 ],
        [ 0.23384207,  0.30219597, -0.23864621,  0.28036636, -0.3695372 ,
          0.3537342 , -0.12034097, -0.03022414, -0.52226186, -0.26620534],
        [ 0.06877089,  0.29128593,  0.52339864,  0.3472799 ,  0.50432336,
          0.2829789 , -0.28482318,  0.4858247 , -0.19799495, -0.47770575],
        [ 0.10424334,  0.11133552, -0.48150936, -0.42249018,  0.49281728,
         -0.4280006 , -0.3307554 ,  0.06750327,  0.174627  , -0.47370154],
        [ 0.49137

# Testing serialization of tf objects

In [11]:
checkpoint = {
    "model_state_dict" : weight_manager.model_state_dict(model),
    "optimizer_state_dict" : optimizer.get_config()
}

In [12]:
with open(pathlib.Path.cwd() / "test_serialization", "wb") as f:
    cloudpickle.dump(checkpoint, f)

In [13]:
with open(pathlib.Path.cwd() / "test_serialization", "rb") as f:
        checkpoint = cloudpickle.load(f)

In [14]:
checkpoint

{'model_state_dict': OrderedDict([('config',
               {'name': 'sequential',
                'layers': [{'class_name': 'InputLayer',
                  'config': {'batch_input_shape': (None, 1),
                   'dtype': 'float32',
                   'sparse': False,
                   'ragged': False,
                   'name': 'dense_input'}},
                 {'class_name': 'Dense',
                  'config': {'name': 'dense',
                   'trainable': True,
                   'dtype': 'float32',
                   'batch_input_shape': (None, 1),
                   'units': 10,
                   'activation': 'linear',
                   'use_bias': True,
                   'kernel_initializer': {'class_name': 'GlorotUniform',
                    'config': {'seed': None}},
                   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
                   'kernel_regularizer': None,
                   'bias_regularizer': None,
                   'activity

In [15]:
optimizer_s = tf.keras.optimizers.serialize(optimizer)

In [16]:
type(optimizer.get_config()), type(optimizer_s)

(dict, dict)

In [17]:
optimizer.get_config(), optimizer_s

({'name': 'Adam',
  'weight_decay': None,
  'clipnorm': None,
  'global_clipnorm': None,
  'clipvalue': None,
  'use_ema': False,
  'ema_momentum': 0.99,
  'ema_overwrite_frequency': None,
  'jit_compile': False,
  'is_legacy_optimizer': False,
  'learning_rate': 0.001,
  'beta_1': 0.9,
  'beta_2': 0.999,
  'epsilon': 1e-07,
  'amsgrad': False},
 {'class_name': 'Custom>Adam',
  'config': {'name': 'Adam',
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'jit_compile': False,
   'is_legacy_optimizer': False,
   'learning_rate': 0.001,
   'beta_1': 0.9,
   'beta_2': 0.999,
   'epsilon': 1e-07,
   'amsgrad': False}})

In [18]:
tf.keras.optimizers.serialize

<function keras.optimizers.serialize(optimizer, use_legacy_format=False)>

In [19]:
model_s_d= tf.keras.layers.serialize(model)

In [20]:
model_s_d

{'class_name': 'Sequential',
 'config': {'name': 'sequential',
  'layers': [{'class_name': 'InputLayer',
    'config': {'batch_input_shape': (None, 1),
     'dtype': 'float32',
     'sparse': False,
     'ragged': False,
     'name': 'dense_input'}},
   {'class_name': 'Dense',
    'config': {'name': 'dense',
     'trainable': True,
     'dtype': 'float32',
     'batch_input_shape': (None, 1),
     'units': 10,
     'activation': 'linear',
     'use_bias': True,
     'kernel_initializer': {'class_name': 'GlorotUniform',
      'config': {'seed': None}},
     'bias_initializer': {'class_name': 'Zeros', 'config': {}},
     'kernel_regularizer': None,
     'bias_regularizer': None,
     'activity_regularizer': None,
     'kernel_constraint': None,
     'bias_constraint': None}},
   {'class_name': 'Dense',
    'config': {'name': 'dense_1',
     'trainable': True,
     'dtype': 'float32',
     'units': 10,
     'activation': 'linear',
     'use_bias': True,
     'kernel_initializer': {'class_

In [21]:
model.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_input'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 1),
    'units': 10,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_init

# Finding how to properly initialize models with config, weights, optimizer, loss, metrics

For a model:
- *config* = its structure (layers' type and dimension)
- *weights* = layers' weights
- *compile config* = its optimizer, loss, metrics and other

We want here to understand how to transfer those caracteristics between model, and how to save and load them.

To get the config with which the model has been ***created***

In [22]:
model.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_input'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 1),
    'units': 10,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_init

In [23]:
model.get_weights()

[array([[-0.49551314,  0.35221773,  0.5255818 ,  0.15151203,  0.44275302,
          0.7328324 ,  0.0546217 ,  0.16471899,  0.22105348,  0.02140474]],
       dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([[ 0.41724896,  0.45191437,  0.340787  ,  0.5463778 , -0.38249683,
         -0.23618022, -0.18816003, -0.42619744, -0.00390798,  0.07460922],
        [-0.15487856,  0.39861256, -0.5139727 , -0.30955482,  0.31888008,
         -0.30110312, -0.4486343 , -0.41079572, -0.44411612,  0.3140511 ],
        [ 0.23384207,  0.30219597, -0.23864621,  0.28036636, -0.3695372 ,
          0.3537342 , -0.12034097, -0.03022414, -0.52226186, -0.26620534],
        [ 0.06877089,  0.29128593,  0.52339864,  0.3472799 ,  0.50432336,
          0.2829789 , -0.28482318,  0.4858247 , -0.19799495, -0.47770575],
        [ 0.10424334,  0.11133552, -0.48150936, -0.42249018,  0.49281728,
         -0.4280006 , -0.3307554 ,  0.06750327,  0.174627  , -0.47370154],
        [ 0.49137

To get the config with which the model has been ***compiled*** : 

In [24]:
type(model.get_compile_config())

NoneType

it has not been compiled yet so it is normal
if we compile it :

In [25]:
model.compile(optimizer=optimizer, loss=loss)

In [26]:
conf= model.get_compile_config()

In [27]:
conf

{'optimizer': {'module': 'keras.optimizers',
  'class_name': 'Adam',
  'config': {'name': 'Adam',
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'jit_compile': False,
   'is_legacy_optimizer': False,
   'learning_rate': 0.0010000000474974513,
   'beta_1': 0.9,
   'beta_2': 0.999,
   'epsilon': 1e-07,
   'amsgrad': False},
  'registered_name': None},
 'loss': {'module': 'keras.losses',
  'class_name': 'SparseCategoricalCrossentropy',
  'config': {'reduction': 'auto',
   'name': 'sparse_categorical_crossentropy',
   'from_logits': True,
   'ignore_class': None},
  'registered_name': None},
 'metrics': None,
 'loss_weights': None,
 'weighted_metrics': None,
 'run_eagerly': None,
 'steps_per_execution': None,
 'jit_compile': None}

In [28]:
conf.keys()

dict_keys(['optimizer', 'loss', 'metrics', 'loss_weights', 'weighted_metrics', 'run_eagerly', 'steps_per_execution', 'jit_compile'])

In [29]:
conf['loss_weights']

Creating a new model

In [30]:
model2 = tf.keras.Sequential()

Compiling it from model's config, we see the differences between configs

In [31]:
model2.compile_from_config(conf)

In [32]:
model2.get_config()

{'name': 'sequential_1', 'layers': []}

In [33]:
model2.get_compile_config()

{'optimizer': {'module': 'keras.optimizers',
  'class_name': 'Adam',
  'config': {'name': 'Adam',
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'jit_compile': False,
   'is_legacy_optimizer': False,
   'learning_rate': 0.0010000000474974513,
   'beta_1': 0.9,
   'beta_2': 0.999,
   'epsilon': 1e-07,
   'amsgrad': False},
  'registered_name': None},
 'loss': {'module': 'keras.losses',
  'class_name': 'SparseCategoricalCrossentropy',
  'config': {'reduction': 'auto',
   'name': 'sparse_categorical_crossentropy',
   'from_logits': True,
   'ignore_class': None},
  'registered_name': None},
 'metrics': None,
 'loss_weights': None,
 'weighted_metrics': None,
 'run_eagerly': None,
 'steps_per_execution': None,
 'jit_compile': None}

In [34]:
model3=tf.keras.Sequential()

In [35]:
model3.from_config(model.get_config())

In [36]:
model3.get_config()

{'name': 'sequential_2', 'layers': []}

In [37]:
model3.compile_from_config(model.get_compile_config())

In [38]:
comp_conf3 = model3.get_compile_config()
comp_conf3

{'optimizer': {'module': 'keras.optimizers',
  'class_name': 'Adam',
  'config': {'name': 'Adam',
   'weight_decay': None,
   'clipnorm': None,
   'global_clipnorm': None,
   'clipvalue': None,
   'use_ema': False,
   'ema_momentum': 0.99,
   'ema_overwrite_frequency': None,
   'jit_compile': False,
   'is_legacy_optimizer': False,
   'learning_rate': 0.0010000000474974513,
   'beta_1': 0.9,
   'beta_2': 0.999,
   'epsilon': 1e-07,
   'amsgrad': False},
  'registered_name': None},
 'loss': {'module': 'keras.losses',
  'class_name': 'SparseCategoricalCrossentropy',
  'config': {'reduction': 'auto',
   'name': 'sparse_categorical_crossentropy',
   'from_logits': True,
   'ignore_class': None},
  'registered_name': None},
 'metrics': None,
 'loss_weights': None,
 'weighted_metrics': None,
 'run_eagerly': None,
 'steps_per_execution': None,
 'jit_compile': None}

Getting optimizer's config from model's compile config

In [42]:
comp_conf3['optimizer']

{'module': 'keras.optimizers',
 'class_name': 'Adam',
 'config': {'name': 'Adam',
  'weight_decay': None,
  'clipnorm': None,
  'global_clipnorm': None,
  'clipvalue': None,
  'use_ema': False,
  'ema_momentum': 0.99,
  'ema_overwrite_frequency': None,
  'jit_compile': False,
  'is_legacy_optimizer': False,
  'learning_rate': 0.0010000000474974513,
  'beta_1': 0.9,
  'beta_2': 0.999,
  'epsilon': 1e-07,
  'amsgrad': False},
 'registered_name': None}

In [39]:
model3.get_config()

{'name': 'sequential_2', 'layers': []}

To instanciate a new model from the config of an existing one, we need to apply `.from_config` to the model class we want to create

In [40]:
model4= tf.keras.Sequential().from_config(model.get_config())

In [41]:
model4.get_config()

{'name': 'sequential',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 1),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'dense_input'}},
  {'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'batch_input_shape': (None, 1),
    'units': 10,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_1',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'linear',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_init

Because we can easily get the optimizer's config with the model's compile config, we could focus on model.